In [1]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os, sys
import django
import pytz

from asgiref.sync import sync_to_async
import pathlib

import os, time
import datetime
import pytz
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

# basic settings

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# indian = pytz.timezone('Asia/Kolkata')

timezone_str = 'Asia/Kolkata'
localtz = pytz.timezone(timezone_str)

# DJANGO INTEGRATION

django_path = "../"

sys.path.append(django_path)
os.environ['DJANGO_SETTINGS_MODULE'] = 'stocks_project.settings'

from historic_data.models import stocks_data
from django_pandas.io import read_frame

# brokerage constants

brokerage_file = '%s/algorithm/%s' % (django_path, "BROKERAGES.json")
with open(brokerage_file) as source:
    BROKERAGES = json.load(source)

# helper functions

def create_timestamp_obj(time_str):
    return localtz.localize(datetime.datetime.strptime(time_str, '%Y-%m-%d'))

def add_moving_average_to_df(df, ma_time):
    df['MA_%s'%(ma_time)] = df.close_price.rolling(window=ma_time).mean()

def cal_slope(unit_value, df, column):
    df['slope_%s'%(column)] = ((df[column] - df[column].shift(unit_value))/unit_value).round(10)
#     df['slope_%s'%(column)] = df[column].shift(-unit_value).apply(lambda x : np.polyfit(np.array(range(unit_value)), x, 1)[0])[4::5]
    
async def get_price_history_df(scrip_name, start_time, end_time):
    start_timestamp = create_timestamp_obj(start_time)
    end_timestamp = create_timestamp_obj(end_time)
    qs_price_history = await sync_to_async(stocks_data.PriceHistory.objects.filter)(timestamp__range=(start_timestamp, end_timestamp), scrip__name=scrip_name)
    qs_price_history.order_by('timestamp')
    df_price_history = read_frame(qs_price_history, fieldnames=['id', 'scrip__name', 'timestamp', 'close_price', 'high_price', 'low_price', 'volume'])
    df_price_history['timestamp'] = df_price_history['timestamp'].dt.tz_convert('Asia/Kolkata').dt.strftime('%Y-%m-%d %H:%M:%S')
    df_price_history = df_price_history.sort_values(by = 'timestamp', ascending = True)
    return df_price_history
    

################################################################################################

# plot graph for given dataframe

def plot_graphs(xAxis, yAxis1, yAxis2):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=xAxis, y=yAxis1,
                        mode='lines',
                        name='close'))

    fig.add_trace(go.Scatter(x=xAxis, y=yAxis2,
                        mode='lines',
                        name='moving average 21'))

    fig.update_layout(  
        xaxis_title=xAxis.name,
        yaxis_title="Closing Price",
        xaxis = dict(type="category", categoryorder='category ascending', showticklabels=False, showgrid=False),
        yaxis = dict(showgrid=False)
        )

    fig.show()

    
# subplots

def subplot_graphs(xAxis, yAxis1, yAxis2):
    
    fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

    fig.add_trace(
        go.Scatter(x=xAxis, y=yAxis1,mode='lines', name='%s vs %s' %(yAxis1.name, xAxis.name)),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=xAxis, y=yAxis2,mode='lines', name='%s vs %s' %(yAxis2.name, xAxis.name)),
        row=2, col=1
    )
    fig.update_xaxes(title_text=xAxis.name, row=1, col=1)
    fig.update_xaxes(title_text=xAxis.name,  row=2, col=1)
    fig.update_yaxes(title_text=yAxis1.name, row=1, col=1)
    fig.update_yaxes(title_text=yAxis2.name, row=2, col=1)
    fig.update_layout(
        height=800, 
        width=1000,
        xaxis = dict(type="category",showgrid=False, showticklabels=False),
        xaxis2 = dict(type="category",showgrid=False, showticklabels=False),
        yaxis = dict(showgrid=False, zeroline=False),
        yaxis2 = dict(showgrid=False)
    )
    fig.show()
    
################################################################################################
    
# title = 'Close Price History Buy / Sell Signals   '

def tick_graph(list_d_df):

#     plt.figure(figsize=(12.2,4.5))
    list_d_df['Buy_Signal_Price'] = list_d_df[list_d_df['buy/sell'] == 'buy']['price']
    list_d_df['Sell_Signal_Price'] = list_d_df[list_d_df['buy/sell'] == 'sell']['price']
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['price'],
                    mode='lines',
                    line=dict(color='deepskyblue', width=2),
                    name='Close Price'))
    fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['ma_9'],
                    mode='lines',
                    line=dict(color='darkorange', width=2),
                    name='Moving Average 9'))
    fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['ma_21'],
                    mode='lines',
                    line=dict(color='hotpink', width=2),
                    name='Moving Average 21'))
    fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['Buy_Signal_Price'],
                    mode='markers',
                    marker=dict(size=10, color="green"),
                    name='Buy Signal'))
    fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['Sell_Signal_Price'],
                    mode='markers',
                    marker=dict(size=10, color="red"),
                    name='Sell Signal'))
    fig.update_layout(
        xaxis_title='Timestamp',
        yaxis_title="Close Price",
        title_text="Close Price History Buy / Sell Signals"
    )

    fig.show()
    
#     plt.scatter(list_d_df['timestamp'], list_d_df['Buy_Signal_Price'], color = 'green', label='Buy Signal', marker = '^', alpha = 1)
#     plt.scatter(list_d_df['timestamp'], list_d_df['Sell_Signal_Price'], color = 'red', label='Sell Signal', marker = 'v', alpha = 1)
#     plt.plot( list_d_df['price'],  label='close price', alpha = 0.35)
#     plt.title(title)
#     plt.xlabel('time stamp',fontsize=18)
#     plt.ylabel('Close Price',fontsize=18)
#     plt.legend( loc='upper right')
#     plt.show()

    
################################################################################################

def calculate_brokerage(side, txn_type, price, broker):
    current_brokerages_dict=BROKERAGES[broker][txn_type]
    brokerages = 0
    for head, value in current_brokerages_dict[side].items():
        brokerages += price*value
    return brokerages

###############################################################################################

# splitting dataframe into sub datframes on the basis of frequency

def split_df_freq(df, frequency):
    no_of_days = len(df)/375
    no_of_sub_df = no_of_days/frequency
    list_df = np.array_split(df, no_of_sub_df)
    return list_df

###############################################################################################
def make_decision_in_stock(
    decision_to_do, 
    current_price,
    qty_stock,
    previous_price,
    transaction_temp_df,
    row,
    cum_profit,
    cum_pab
#     slope
            ):
    
    brokerage = calculate_brokerage(decision_to_do, "intraday", current_price, "zerodha")
                        
    if qty_stock == 0:
        profit = 0
        pab = 0 - brokerage
        profit_percentage = 0

    else:
        if decision_to_do == 'buy': 
            profit = previous_price - current_price
        elif decision_to_do == 'sell': 
            profit = current_price - previous_price
        pab = profit - brokerage
        profit_percentage = float(profit/previous_price) * 100


    previous_price = current_price
    cum_profit += profit
    cum_pab += pab 
    
    if decision_to_do == 'buy': 
        qty_stock += 1
    elif decision_to_do == 'sell': 
        qty_stock -= 1
    
    decision = 'nothing'
    
    transaction_temp_df = transaction_temp_df.append({
        'scrip' : row['scrip__name'],
        'timestamp':row['timestamp'],
        'buy/sell': decision_to_do,
        'price':current_price,
        'profit':profit,
        'brokerage':brokerage,
        'qtystock': qty_stock,
        'cum_profit':cum_profit,
        'pab':pab,
        'cum_pab':cum_pab,
#         'profit_percentage': profit_percentage,
#         'slope': slope
    }, ignore_index=True)
    
#     return profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, profit_percentage, decision, transaction_temp_df
    return profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_temp_df


##################################################################

def cum_pab_graph(list_timestamp, list_cumpab):
    data = [go.Bar(
       x = list_timestamp,
       y = list_cumpab
    )]
    fig = go.Figure(data=data)
    fig.update_layout(
        title="Cumulative profit after brokerage for selected days on RELIANCE",
        xaxis_title="Timestamp",
        yaxis_title="Cumulative profit after brokerage")
    iplot(fig)



In [2]:
avg_sloss_buff_dec_df = pd.DataFrame(columns = ['Stop loss', 'Buffer', 'Total cumulative PAB', 'Average cumulative PAB', 'Average percentage cumulative PAB'])

In [3]:
# GOAL -------------->>>>>>>
# **************************************  INCREASE CUM PAB VALUE  *******************************************************************88

def goldencross_transaction_table(list_ph_df, sloss_dec, buff_dec, sloss_buff_dec_df, list_cumpab, list_timestamp):    
# def transaction_stop_loss_test_cases(test_case_number, scrip, start_time, end_time):    
    
    list_t_df = []
    list_d_df = []
    
    for df in list_ph_df:
        
        profit = 0
        cum_profit = 0
        qty_stock = 0
        pab = 0
        cum_pab = 0   
        beg_time = 16
        profit_percentage = 0
        buffer_value = buff_dec
        brokerage = 0
        decision = 'nothing'
        decision_to_do = 'nothing'
        previous_price = 1
        difference_price = 0
        previous_closing_price = 0
        stop_loss_price = 0
        stop_loss_close_price = 0;
        stop_loss_percent = sloss_dec
        apply_stop_loss = 'nothing'

        transaction_df = pd.DataFrame(columns=
                                  [
                                   'scrip',
                                   'timestamp',
                                   'buy/sell',
                                   'price',
                                   'brokerage',
                                   'qtystock',
                                   'profit',
                                   'cum_profit',
                                   'pab',
                                   'cum_pab',
    #                                'profit_percentage',
    #                                'slope'
                                  ]
                                 )
        
        dummy_df = pd.DataFrame(columns=
                              [
                               'scrip',
                               'timestamp',
                               'buy/sell',
                               'price',
                               'brokerage',
                               'qtystock',
                               'profit',
                               'cum_profit',
                               'pab',
                               'cum_pab',
                                'ma_9',
                                'ma_21'
                              ]
                             )


        for index, row in  df[beg_time:].iterrows():

            current_price = row['close_price']
            previous_closing_price = df['close_price'][index-1]
            difference_price = current_price - previous_closing_price

            if (row["timestamp"] >= df["timestamp"].iloc[-30]):
                if qty_stock == 0:
#                 if (len(transaction_df)%2)==0:
                    break
                else:
                    if (row["timestamp"] == df["timestamp"].iloc[-1]):
                        if qty_stock == -1:
                            decision_to_do = 'buy'
                            profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_df = make_decision_in_stock("buy", current_price, qty_stock, previous_price, transaction_df, row, cum_profit, cum_pab)
                        elif qty_stock == 1:
                            decision_to_do = 'sell'
                            profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_df = make_decision_in_stock("sell", current_price, qty_stock, previous_price, transaction_df, row, cum_profit, cum_pab) 
                        dummy_df = dummy_df.append({
                            'scrip' : row['scrip__name'],
                            'timestamp':row['timestamp'],
                            'buy/sell': decision_to_do,
                            'price':current_price,
                            'profit':profit,
                            'brokerage':brokerage,
                            'qtystock': qty_stock,
                            'cum_profit':cum_profit,
                            'pab':pab,
                            'cum_pab':cum_pab,
                            'ma_9': row['MA_9'],
                            'ma_21': row['MA_21']
                        }, ignore_index=True)
                        break
                    
           
                
            if apply_stop_loss == 'buy' and difference_price > 0 and current_price > stop_loss_close_price:
                stop_loss_price += difference_price
                stop_loss_close_price = current_price
   
            elif apply_stop_loss == 'sell' and difference_price < 0 and current_price < stop_loss_close_price:
                stop_loss_price -= abs(difference_price)
                stop_loss_close_price = current_price
               
            if apply_stop_loss == 'buy' and current_price < stop_loss_price and qty_stock != -1:
                apply_stop_loss = 'nothing'
                stop_loss_price = 0
                decision_to_do = 'sell'
                profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_df = make_decision_in_stock("sell", current_price, qty_stock, previous_price, transaction_df, row, cum_profit, cum_pab) 

            elif apply_stop_loss == 'sell' and current_price > stop_loss_price and qty_stock != 1:
                apply_stop_loss = 'nothing'
                stop_loss_price = 0
                decision_to_do = 'buy'
                profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_df = make_decision_in_stock("buy", current_price, qty_stock, previous_price, transaction_df, row, cum_profit, cum_pab)

            if decision == 'buy' and ((row['MA_9'] - row['MA_21']) / row['MA_9'])>buffer_value :
                stop_loss_price = current_price - stop_loss_percent * current_price
                stop_loss_close_price = current_price
                decision_to_do = 'buy'
                profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_df = make_decision_in_stock("buy", current_price, qty_stock, previous_price, transaction_df, row, cum_profit, cum_pab)
                apply_stop_loss = 'buy'

            elif decision == 'sell' and ((row['MA_21'] - row['MA_9']) / row['MA_21'])>buffer_value : 
                stop_loss_price = current_price + stop_loss_percent * current_price
                stop_loss_close_price = current_price
                decision_to_do = 'sell'
                profit, previous_price, brokerage, qty_stock, cum_profit, pab, cum_pab, decision, transaction_df = make_decision_in_stock("sell", current_price, qty_stock, previous_price, transaction_df, row, cum_profit, cum_pab) 
                apply_stop_loss = 'sell'

            if row['MA_9'] >= row['MA_21'] and df['MA_9'][index-1] <= df['MA_21'][index-1] and qty_stock != 1:
                decision = 'buy'

            elif row['MA_9'] <= row['MA_21'] and df['MA_9'][index-1] >= df['MA_21'][index-1] and qty_stock != -1:
                decision = 'sell'
                
            dummy_df = dummy_df.append({
                'scrip' : row['scrip__name'],
                'timestamp':row['timestamp'],
                'buy/sell': decision_to_do,
                'price':current_price,
                'profit':profit,
                'brokerage':brokerage,
                'qtystock': qty_stock,
                'cum_profit':cum_profit,
                'pab':pab,
                'cum_pab':cum_pab,
                'ma_9': row['MA_9'],
                'ma_21': row['MA_21']
            }, ignore_index=True)
            
            if decision_to_do != 'nothing':
                decision_to_do = 'nothing'
                
#             print(d.time() for d in row["timestamp"])
                
#             if (row["timestamp"] == df["timestamp"].iloc[-30]):
            
                
        if len(transaction_df)>0:
            
            per_cum_pab = ((cum_pab/transaction_df['price'].iloc[-1]) * 100)
            
            sloss_buff_dec_df = sloss_buff_dec_df.append({
                'sloss_dec' : sloss_dec,
                'buff_dec' : buff_dec,
                'timestamp': df['timestamp'].iloc[0],
                'cum_pab' : cum_pab,
                'per_cum_pab' : per_cum_pab
            }, ignore_index=True)
            
            list_cumpab.append(cum_pab)    
            list_timestamp.append(df['timestamp'].iloc[0])
            list_t_df.append(transaction_df)
            list_d_df.append(dummy_df)
#         tick_graph(dummy_df)
            
#         else:
#             per_cum_pab = 0
                
#         sloss_buff_dec_df = sloss_buff_dec_df.append({
#             'sloss_dec' : sloss_dec,
#             'buff_dec' : buff_dec,
#             'timestamp': df['timestamp'].iloc[0],
#             'cum_pab' : cum_pab,
#             'per_cum_pab' : per_cum_pab
#         }, ignore_index=True)
        
#         list_cumpab.append(cum_pab)    
#         list_timestamp.append(df['timestamp'].iloc[0])
#         list_t_df.append(transaction_df)
    
    return list_t_df, list_d_df, sloss_buff_dec_df, list_cumpab, list_timestamp

In [4]:
# make transaction table (dataframe) and return it
# return list of transaction tables

async def transaction_table(scrip_name, start_time, end_time, frequency, sloss_dec, buff_dec, sloss_buff_dec_df, list_cumpab, list_timestamp, extra_param, algo='goldencross'):

    list_ph_df = []
    list_t_df = []
    list_d_df = []
    
    if algo == 'goldencross':
        ma1 = extra_param['ma1']
        ma2 = extra_param['ma2']
        price_history_df = await get_price_history_df(scrip_name, start_time, end_time)
        add_moving_average_to_df(price_history_df, ma1)
        add_moving_average_to_df(price_history_df, ma2)
#         cal_slope(extra_param['slope_unit'], price_history_df, 'MA_%s' %slope )
        list_ph_df = split_df_freq(price_history_df, frequency)
        list_t_df, list_d_df, sloss_buff_dec_df, list_cumpab, list_timestamp = goldencross_transaction_table(list_ph_df, sloss_dec, buff_dec, sloss_buff_dec_df, list_cumpab, list_timestamp)
#         list_t_df = slope_transaction_table(list_ph_df, slope)
                
    return list_ph_df, list_t_df, list_d_df, sloss_buff_dec_df, list_cumpab, list_timestamp

In [5]:
#### if you give a time for which there is no data, then it gives a 'Can only use .dt accessor with datetimelike values' error

In [6]:
scrip_name = "RELIANCE"
start_timestamp = "2020-06-01"
end_timestamp = "2020-08-27"
frequency = 1
extra_param = {
    'ma1': 9,
    'ma2' : 21
}
# cum_pab_list = []

# j -> stop loss (0.01 to 0.035, 0.005)
# i -> buffer values (0.0008 to 0.0028, 0.0002)


for j in np.arange(0.01, 0.035, 0.005):
    for i in np.arange(0.0008, 0.0028, 0.0002):
# for j in np.arange(0.025, 0.0251, 0.005):
#     for i in np.arange(0.0022, 0.002210, 0.0002):
        list_cumpab = []
        list_timestamp = []
        sloss_buff_dec_df = pd.DataFrame(columns = ['sloss_dec', 'buff_dec', 'timestamp', 'cum_pab', 'per_cum_pab'])
        list_ph_df, list_t_df, list_d_df, sloss_buff_dec_df, list_cumpab, list_timestamp = await transaction_table(scrip_name, start_timestamp, end_timestamp, frequency, j, i, sloss_buff_dec_df, list_cumpab, list_timestamp, extra_param, 'goldencross')      
#         cum_pab_graph(list_timestamp, list_cumpab)
#         tick_graph(list_d_df[0])
        avg_cum_pab = sloss_buff_dec_df['cum_pab'].mean()
        sum_cum_pab = sloss_buff_dec_df['cum_pab'].sum()
        avg_per_cum_pab = sloss_buff_dec_df['per_cum_pab'].mean()
        avg_sloss_buff_dec_df = avg_sloss_buff_dec_df.append({
            'Stop loss' : j,
            'Buffer': i,
            'Total cumulative PAB' : sum_cum_pab,
            'Average cumulative PAB' : avg_cum_pab,
            'Average percentage cumulative PAB' : avg_per_cum_pab
        }, ignore_index=True)
# list_ph_df, list_t_df = await transaction_table(scrip_name, start_timestamp, end_timestamp, frequency, extra_param, 'slope')

In [7]:
# tick_graph(list_d_df[0])
# for df in list_t_df:
#     print(df.to_string(index = False))
#     print("\n \n")

In [8]:
avg_sloss_buff_dec_df.sort_values(by = ['Total cumulative PAB'], ascending=False)

,Stop loss,Buffer,Total cumulative PAB,Average cumulative PAB,Average percentage cumulative PAB
37,0.025,0.0022,93.808229,1.839377,0.105479
38,0.025,0.0024,86.178797,1.795392,0.104630
47,0.030,0.0022,79.611070,1.561001,0.090034
27,0.020,0.0022,73.244808,1.436173,0.091576
48,0.030,0.0024,71.981638,1.499617,0.088229
28,0.020,0.0024,67.166031,1.399292,0.091818
39,0.025,0.0026,-12.618855,-0.274323,0.001053
36,0.025,0.0020,-22.096863,-0.409201,-0.011719
29,0.020,0.0026,-29.036544,-0.631229,-0.008430
46,0.030,0.0020,-29.898137,-0.553669,-0.019938


In [9]:
# avg_sloss_buff_dec_df.to_csv('sloss_buff_dec.csv')

In [10]:
# def tick_graph(list_d_df):

# #     plt.figure(figsize=(12.2,4.5))
#     list_d_df['Buy_Signal_Price'] = list_d_df[list_d_df['buy/sell'] == 'buy']['price']
#     list_d_df['Sell_Signal_Price'] = list_d_df[list_d_df['buy/sell'] == 'sell']['price']
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['price'],
#                     mode='lines',
#                     line=dict(color='#B8860B', width=2),
#                     name='Close Price'))
#     fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['ma_9'],
#                     mode='lines',
#                     line=dict(color='#1E90FF', width=2),
#                     name='Moving average 9'))
#     fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['ma_21'],
#                     mode='lines',
#                     line=dict(color='#FF1493', width=2),
#                     name='Moving average 21'))
#     fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['Buy_Signal_Price'],
#                     mode='markers',
#                     marker=dict(size=12, color="#DC143C", symbol = 'triangle-up'),
#                     name='Buy Signal'))
#     fig.add_trace(go.Scatter(x=list_d_df['timestamp'], y=list_d_df['Sell_Signal_Price'],
#                     mode='markers',
#                     marker=dict(size=12, color="#556B2F", symbol = 'triangle-up'),
#                     name='Sell Signal'))
#     fig.update_layout(
#         xaxis_title='Timestamp',
#         yaxis_title="Close Price",
#         title_text="Close Price History Buy / Sell Signals"
#     )

#     fig.show()
# tick_graph(list_d_df[0])
# for df in list_t_df:
#     print(df.to_string(index = False))
#     print("\n \n")

In [11]:
# for df in list_d_df:
#     print(tick_graph(df))

In [12]:
# for df in list_t_df:
#     print(df.to_string(index = False))
#     print("\n \n")

In [13]:
# y = f(slope, slope_diff_unit_value, decision_consistent_time)